In [43]:
import pandas as pd
import os
from datetime import datetime, timedelta
import numpy as np
import math

In [44]:
sp500_url = "https://raw.githubusercontent.com/saikr789/stock-index-risk/master/SP500companies.csv"

In [45]:
# sp500 = pd.read_csv(sp500_url)
# sp500.to_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500companies.csv", index=False)
sp500 = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500companies.csv")
sp500_sc = list(sp500['Security Code'])
sp500_sn = list(sp500['Security Name'])
sp500_companies = {str(sp500_sc[i]) : sp500_sn[i] for i in range(len(sp500_sc))}
# sp500_companies

In [46]:
def calculate_profit(data, end_date, days_to_search, investment, final_df, final_df1, sec_num):
    start_date = end_date - timedelta(days = days_to_search)
#     print(start_date, end_date)
    mask = (data["date"] >= start_date) & (data["date"] <= end_date)
    data = data.loc[mask]
    data = data.reset_index(drop=True)
    actual_inv = investment
    actual_shares = 0
    pred_inv = investment
    pred_shares = 0
    i = 0
    inv_status = False
    ext_status = False
    actual_first_inv = 0
    pred_first_inv = 0
    pred_ub = 0
    while (i <= data.shape[0] - 1):
#         print(data.loc[i, "date"])
        if (not inv_status):
            if (data.loc[i, "invest"]):
                actual_shares = actual_inv // data.loc[i, "close"]
                actual_first_inv = actual_inv
                actual_inv = 0
                pred_shares = pred_inv // data.loc[i, "close"]
                pred_first_inv = pred_inv
                pred_inv = 0
                pred_ub = data.loc[i, "predicted ub"] * data.loc[i, "close"]
                inv_status = True
                ext_status = False
                row = {}
                row["Date"] = data.loc[i, "date"]
                row["Close Price"] = data.loc[i, "close"]
                row["Invest/Exit"] = "INVEST"
                row["Actual Investment"] = actual_inv
                row["Actual Shares"] = actual_shares
                row["Actual Returns %"] = 0
                row["Predicted Investment"] = pred_inv
                row["Predicted Shares"] = pred_shares
                row["Predicted Value-UB"] = pred_ub
                row["Predicted Returns %"] = 0
                final_df = final_df.append(row, ignore_index=True)
        else:
            if (data.loc[i, "exit"]):
                actual_inv = actual_shares * data.loc[i, "close"]
                actual_shares = 0
                pred_inv = pred_shares * pred_ub
                pred_shares = 0
                ext_status = True
                inv_status = False
                actual_avg = (actual_inv - actual_first_inv) / actual_first_inv
                pred_avg = (pred_inv - pred_first_inv) / pred_first_inv
                row = {}
                row["Date"] = data.loc[i, "date"]
                row["Close Price"] = data.loc[i, "close"]
                row["Invest/Exit"] = "EXIT"
                
                row["Actual Investment"] = actual_inv
                row["Actual Shares"] = actual_shares
                row["Actual Returns %"] = actual_avg
                row["Predicted Investment"] = pred_inv
                row["Predicted Shares"] = pred_shares
                row["Predicted Value-UB"] = 0
                row["Predicted Returns %"] = pred_avg
                
                final_df = final_df.append(row, ignore_index=True)
        i += 1
        
    
    if ((not inv_status) and (not ext_status)):
        row = {}
        row["Date"] = data.loc[data.shape[0] - 1, "date"]
        row["Close Price"] = data.loc[data.shape[0] - 1, "close"]
        row["Invest/Exit"] = "-"
        
        row["Actual Investment"] = 0
        row["Actual Shares"] = 0
        row["Actual Returns %"] = 0
        row["Predicted Investment"] = 0
        row["Predicted Shares"] = 0
        row["Predicted Value-UB"] = 0
        row["Predicted Returns %"] = 0
        
        final_df = final_df.append(row, ignore_index=True)
    
    if (inv_status and (not ext_status)):
        actual_inv = actual_shares * data.loc[data.shape[0] - 1, "close"]
        actual_avg = (actual_inv - actual_first_inv) / actual_first_inv
        actual_shares = 0
        pred_inv = pred_shares * pred_ub
        pred_avg = (pred_inv - pred_first_inv) / pred_first_inv
        pred_shares = 0
        row = {}
        row["Date"] = data.loc[data.shape[0] - 1, "date"]
        row["Close Price"] = data.loc[data.shape[0] - 1, "close"]
        row["Invest/Exit"] = "-"
        
        
        row["Actual Investment"] = actual_inv
        row["Actual Shares"] = actual_shares
        row["Actual Returns %"] = actual_avg
        row["Predicted Investment"] = pred_inv
        row["Predicted Shares"] = pred_shares
        row["Predicted Value-UB"] = pred_ub
        row["Predicted Returns %"] = pred_avg
        
        final_df = final_df.append(row, ignore_index=True)
    
    
    print(final_df)
    
    actual_total_avg = final_df['Actual Returns %'].sum() / final_df[final_df["Invest/Exit"].isin(["EXIT", "-"])].shape[0]
    pred_total_avg = final_df['Predicted Returns %'].sum() / final_df[final_df["Invest/Exit"].isin(["EXIT", "-"])].shape[0]
    pred_actual_diff = pred_total_avg - actual_total_avg
    company = sp500_companies[sec_num]
    r = {}
    r["Company"] = company
    r["Security Id"] = sec_num
    r["Days"] = days_to_search
    r["Actual average returns %"] = actual_total_avg
    r["Predicted average returns %"] = pred_total_avg
    r["Difference"] = pred_actual_diff
#     r["Days"] = days_to_search
    final_df1 = final_df1.append(r, ignore_index=True)
    
    print(final_df1)
    
    return actual_inv, actual_shares, pred_inv, pred_shares, final_df, final_df1
        
            

In [47]:
path = os.getcwd()
path

'C:\\Users\\venu\\Desktop\\Stock Market Analysis'

In [48]:
%%time


def get_simulation_loop(days):
    columns1 = ["Company", "Security Id", "Days", "Actual average returns %", "Predicted average returns %", "Difference"]
    fd_df1 = pd.DataFrame(columns = columns1)
    
    for filename in sp500_sc:
#     for filename in os.listdir(os.path.join(path,"Data/SP500_simulation_input_files/simulation")):
        print(filename)
        if (filename != 500290):
            continue
        try :
    #         columns = ["Date", "Close Price", "Invest/Exit", "Actual Investment", "Actual Shares", "Actual Returns %", "Predicted Investment", "Predicted Shares", "Predicted UB", "Predicted Returns %"]
    #         fd_df = pd.DataFrame(columns = columns)
            print("-------------------------------------------------------")
            name = str(filename)
#             name = filename[ : 6]
            print(name)
            df1 = pd.read_csv("https://raw.githubusercontent.com/saikr789/stock-analysis-tool-1011-data/master/Data/Simulation/" + str(filename) + "_" + str(days) + ".csv")
#             df1 = pd.read_csv("https://raw.githubusercontent.com/saikr789/stock-index-risk/master/Data/Simulation/" + str(filename) + ".csv")
#             df1 = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500_simulation_input_files\\simulation\\" + filename)
            df1["date"] = pd.to_datetime(df1["date"])
            st_date = df1.loc[0, "date"] + timedelta(days = days)
            j = 1
            
            print("before while", st_date, df1.loc[df1.shape[0] - j, "date"])
            
            while (st_date <= (df1.loc[df1.shape[0] - j, "date"])):
                columns = ["Date", "Close Price", "Invest/Exit", "Actual Investment", "Actual Shares", "Actual Returns %", "Predicted Investment", "Predicted Shares", "Predicted Value-UB", "Predicted Returns %"]
                fd_df = pd.DataFrame(columns = columns)
                ed_date = df1.loc[df1.shape[0] - j, "date"]
#                 print(st_date, ed_date, "while")
                inst = 100000
                actual_investment, actual_shares, predicted_investment, predicted_shares, final_df, fd_df1 = calculate_profit(df1, ed_date, days, inst, fd_df, fd_df1, name)
    #             print(actual_investment, actual_shares)
    #             print(predicted_investment, predicted_shares)
        #         print(final_df)
#                 print(fd_df1)
    #             final_df.to_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500_simulation_results\\" + name + "_" + str(days) + "_days_" + str(j) + ".csv", index = False)
    #             print("-------------------------------------------------------")
                j += 1
#             break
            print("-------------------------------------------------------")
        except Exception as e:
            print(e)
            continue

    fd_df1 = fd_df1.sort_values(by = ['Security Id'], ascending = [True])
    print(fd_df1)
#     fd_df1.to_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500_simulation_results\\all_" + str(days) + "days_Results.csv", index = False)



Wall time: 0 ns


In [49]:
%%time

days = [30, 60, 90, 180, 360, 720, 1080]

for d in days:
    if(d != 60):
        continue
    print(d)
    print("*****************************************************************")
    get_simulation_loop(d)
    print("*****************************************************************")
#     break

60
*****************************************************************
500002
500003
500008
500010
500020
500027
500031
500033
500034
500038
500040
500042
500043
500048
500049
500067
500084
500085
500086
500087
500092
500096
500103
500104
500106
500109
500110
500112
500113
500114
500116
500124
500125
500126
500133
500135
500144
500148
500163
500164
500165
500171
500180
500182
500183
500184
500188
500209
500228
500233
500238
500247
500249
500251
500252
500253
500257
500260
500265
500266
500271
500285
500290
-------------------------------------------------------
500290
before while 2017-05-23 00:00:00 2021-03-26 00:00:00
        Date  Close Price Invest/Exit Actual Investment Actual Shares  \
0 2021-03-26     82031.75           -                 0             0   

  Actual Returns % Predicted Investment Predicted Shares Predicted Value-UB  \
0                0                    0                0                  0   

  Predicted Returns %  
0                   0  
    Company Security

     Company Security Id Days  Actual average returns %  \
0   MRF Ltd.      500290   60                       0.0   
1   MRF Ltd.      500290   60                       0.0   
2   MRF Ltd.      500290   60                       0.0   
3   MRF Ltd.      500290   60                       0.0   
4   MRF Ltd.      500290   60                       0.0   
5   MRF Ltd.      500290   60                       0.0   
6   MRF Ltd.      500290   60                       0.0   
7   MRF Ltd.      500290   60                       0.0   
8   MRF Ltd.      500290   60                       0.0   
9   MRF Ltd.      500290   60                       0.0   
10  MRF Ltd.      500290   60                       0.0   
11  MRF Ltd.      500290   60                       0.0   
12  MRF Ltd.      500290   60                       0.0   

    Predicted average returns %  Difference  
0                           0.0         0.0  
1                           0.0         0.0  
2                           0.0    

        Date  Close Price Invest/Exit Actual Investment Actual Shares  \
0 2021-02-26      84470.3           -                 0             0   

  Actual Returns % Predicted Investment Predicted Shares Predicted Value-UB  \
0                0                    0                0                  0   

  Predicted Returns %  
0                   0  
     Company Security Id Days  Actual average returns %  \
0   MRF Ltd.      500290   60                       0.0   
1   MRF Ltd.      500290   60                       0.0   
2   MRF Ltd.      500290   60                       0.0   
3   MRF Ltd.      500290   60                       0.0   
4   MRF Ltd.      500290   60                       0.0   
5   MRF Ltd.      500290   60                       0.0   
6   MRF Ltd.      500290   60                       0.0   
7   MRF Ltd.      500290   60                       0.0   
8   MRF Ltd.      500290   60                       0.0   
9   MRF Ltd.      500290   60                       0.0 

     Company Security Id Days  Actual average returns %  \
0   MRF Ltd.      500290   60                       0.0   
1   MRF Ltd.      500290   60                       0.0   
2   MRF Ltd.      500290   60                       0.0   
3   MRF Ltd.      500290   60                       0.0   
4   MRF Ltd.      500290   60                       0.0   
5   MRF Ltd.      500290   60                       0.0   
6   MRF Ltd.      500290   60                       0.0   
7   MRF Ltd.      500290   60                       0.0   
8   MRF Ltd.      500290   60                       0.0   
9   MRF Ltd.      500290   60                       0.0   
10  MRF Ltd.      500290   60                       0.0   
11  MRF Ltd.      500290   60                       0.0   
12  MRF Ltd.      500290   60                       0.0   
13  MRF Ltd.      500290   60                       0.0   
14  MRF Ltd.      500290   60                       0.0   
15  MRF Ltd.      500290   60                       0.0 

30                          0.0         0.0  
        Date  Close Price Invest/Exit Actual Investment Actual Shares  \
0 2021-02-08      90810.1           -                 0             0   

  Actual Returns % Predicted Investment Predicted Shares Predicted Value-UB  \
0                0                    0                0                  0   

  Predicted Returns %  
0                   0  
     Company Security Id Days  Actual average returns %  \
0   MRF Ltd.      500290   60                       0.0   
1   MRF Ltd.      500290   60                       0.0   
2   MRF Ltd.      500290   60                       0.0   
3   MRF Ltd.      500290   60                       0.0   
4   MRF Ltd.      500290   60                       0.0   
5   MRF Ltd.      500290   60                       0.0   
6   MRF Ltd.      500290   60                       0.0   
7   MRF Ltd.      500290   60                       0.0   
8   MRF Ltd.      500290   60                       0.0   
9   MRF Lt

...

      Company Security Id Days  Actual average returns %  \
0    MRF Ltd.      500290   60                       0.0   
1    MRF Ltd.      500290   60                       0.0   
2    MRF Ltd.      500290   60                       0.0   
3    MRF Ltd.      500290   60                       0.0   
4    MRF Ltd.      500290   60                       0.0   
..        ...         ...  ...                       ...   
855  MRF Ltd.      500290   60                       0.0   
856  MRF Ltd.      500290   60                       0.0   
857  MRF Ltd.      500290   60                       0.0   
858  MRF Ltd.      500290   60                       0.0   
859  MRF Ltd.      500290   60                       0.0   

     Predicted average returns %  Difference  
0                            0.0         0.0  
1                            0.0         0.0  
2                            0.0         0.0  
3                            0.0         0.0  
4                            0.0         0.0

0                   0  
      Company Security Id Days  Actual average returns %  \
0    MRF Ltd.      500290   60                       0.0   
1    MRF Ltd.      500290   60                       0.0   
2    MRF Ltd.      500290   60                       0.0   
3    MRF Ltd.      500290   60                       0.0   
4    MRF Ltd.      500290   60                       0.0   
..        ...         ...  ...                       ...   
861  MRF Ltd.      500290   60                       0.0   
862  MRF Ltd.      500290   60                       0.0   
863  MRF Ltd.      500290   60                       0.0   
864  MRF Ltd.      500290   60                       0.0   
865  MRF Ltd.      500290   60                       0.0   

     Predicted average returns %  Difference  
0                            0.0         0.0  
1                            0.0         0.0  
2                            0.0         0.0  
3                            0.0         0.0  
4                   

      Company Security Id Days  Actual average returns %  \
0    MRF Ltd.      500290   60                       0.0   
1    MRF Ltd.      500290   60                       0.0   
2    MRF Ltd.      500290   60                       0.0   
3    MRF Ltd.      500290   60                       0.0   
4    MRF Ltd.      500290   60                       0.0   
..        ...         ...  ...                       ...   
866  MRF Ltd.      500290   60                       0.0   
867  MRF Ltd.      500290   60                       0.0   
868  MRF Ltd.      500290   60                       0.0   
869  MRF Ltd.      500290   60                       0.0   
870  MRF Ltd.      500290   60                       0.0   

     Predicted average returns %  Difference  
0                            0.0         0.0  
1                            0.0         0.0  
2                            0.0         0.0  
3                            0.0         0.0  
4                            0.0         0.0

[877 rows x 6 columns]
        Date  Close Price Invest/Exit Actual Investment Actual Shares  \
0 2017-05-26     65391.35           -                 0             0   

  Actual Returns % Predicted Investment Predicted Shares Predicted Value-UB  \
0                0                    0                0                  0   

  Predicted Returns %  
0                   0  
      Company Security Id Days  Actual average returns %  \
0    MRF Ltd.      500290   60                       0.0   
1    MRF Ltd.      500290   60                       0.0   
2    MRF Ltd.      500290   60                       0.0   
3    MRF Ltd.      500290   60                       0.0   
4    MRF Ltd.      500290   60                       0.0   
..        ...         ...  ...                       ...   
873  MRF Ltd.      500290   60                       0.0   
874  MRF Ltd.      500290   60                       0.0   
875  MRF Ltd.      500290   60                       0.0   
876  MRF Ltd.      5002

      Company Security Id Days  Actual average returns %  \
0    MRF Ltd.      500290   60                       0.0   
580  MRF Ltd.      500290   60                       0.0   
581  MRF Ltd.      500290   60                       0.0   
582  MRF Ltd.      500290   60                       0.0   
583  MRF Ltd.      500290   60                       0.0   
..        ...         ...  ...                       ...   
297  MRF Ltd.      500290   60                       0.0   
298  MRF Ltd.      500290   60                       0.0   
299  MRF Ltd.      500290   60                       0.0   
288  MRF Ltd.      500290   60                       0.0   
880  MRF Ltd.      500290   60                       0.0   

     Predicted average returns %  Difference  
0                            0.0         0.0  
580                          0.0         0.0  
581                          0.0         0.0  
582                          0.0         0.0  
583                          0.0         0.0